In [31]:
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np

from api_logs import parse_log

In [32]:
log_file_name = '/aidata/junjie/repo/gitee/yj-anls/data/task_asr_evaluate/docker_logs_anls/230912/17-8309.log'
df_rec, df_task = parse_log(log_file_name)

start_time =  pd.Timestamp('2023-09-12 13:20:01')
df_rec = df_rec[df_rec['time'] > start_time]
df_task = df_task[df_task['time'] > start_time]
df_task

,task_id,enter_time,complete_time,time
12442,2302297160166334464,2023-09-12 13:20:05,2023-09-12 13:22:27,2023-09-12 13:20:05
12446,2302297165157556224,2023-09-12 13:20:06,2023-09-12 13:22:32,2023-09-12 13:20:06
12447,2302297183193063424,2023-09-12 13:20:08,2023-09-12 13:22:39,2023-09-12 13:20:08
12443,2302297186166669312,2023-09-12 13:20:05,2023-09-12 13:22:27,2023-09-12 13:20:05
12378,2302297217552801792,2023-09-12 13:20:12,2023-09-12 13:20:12,2023-09-12 13:20:12
...,...,...,...,...
13487,2302317159253729280,2023-09-12 13:59:49,2023-09-12 13:59:52,2023-09-12 13:59:49
13491,2302317225305628672,2023-09-12 13:59:57,2023-09-12 14:00:08,2023-09-12 13:59:57
13493,2302317260059631616,2023-09-12 14:00:01,2023-09-12 14:00:08,2023-09-12 14:00:01
13495,2302317262467162112,2023-09-12 14:00:01,2023-09-12 14:00:09,2023-09-12 14:00:01


## 请求分布

In [33]:
df_task_p = df_task.copy()
df_task_p.index = pd.to_datetime(df_task_p['time'])
df_task_p = df_task_p.resample('60s').count()
df_task_p['time'] = df_task_p.index
print(df_task_p)
# df_task_p = df_task_p[(df_task_p['time'].dt.hour >= 8) & (df_task_p['time'].dt.hour < 20)]


# 绘制直线图
fig = px.line(x=df_task_p.index, y=df_task_p['task_id'])
fig.update_layout(
    title=f'{df_task_p.index.min()}至{df_task_p.index.max()} 每分钟请求数',
    xaxis_title='时间',
    yaxis_title='每分钟请求数'
)
# 显示图形
fig.show()

df_task_cout = df_task_p['task_id'].value_counts()
print(df_task_cout)
df_task_cout = pd.DataFrame(df_task_cout).sort_index()
# df_task_cout = df_task_cout.drop(index=0)
df_task_cout = df_task_cout.reset_index()
df_task_cout.columns = ['task_num_per_n_sec', 'count']
df_task_cout['percent'] = df_task_cout['count']/df_task_cout['count'].sum()*100


# 绘制条形图
fig = px.bar(df_task_cout, x='task_num_per_n_sec', y='count', text='percent')
fig.update_traces(texttemplate='%{text:.2f}%', textposition='outside')
fig.update_layout(
    title=f'{df_task_p.index.min()}至{df_task_p.index.max()} 每5s请求数分布',
    xaxis_title='每5s请求数',
    yaxis_title='数量占比'
)
fig.show()

                     task_id  enter_time  complete_time                time
time                                                                       
2023-09-12 13:20:00       31          31             31 2023-09-12 13:20:00
2023-09-12 13:21:00       29          29             29 2023-09-12 13:21:00
2023-09-12 13:22:00       28          28             28 2023-09-12 13:22:00
2023-09-12 13:23:00       31          31             31 2023-09-12 13:23:00
2023-09-12 13:24:00       28          28             28 2023-09-12 13:24:00
2023-09-12 13:25:00       28          28             28 2023-09-12 13:25:00
2023-09-12 13:26:00       28          28             28 2023-09-12 13:26:00
2023-09-12 13:27:00       28          28             28 2023-09-12 13:27:00
2023-09-12 13:28:00       29          29             29 2023-09-12 13:28:00
2023-09-12 13:29:00       28          28             28 2023-09-12 13:29:00
2023-09-12 13:30:00       29          29             29 2023-09-12 13:30:00
2023-09-12 1

task_id
28    16
29     6
30     5
31     4
32     4
26     1
18     1
6      1
4      1
10     1
3      1
Name: count, dtype: int64


## rec时间分布-01-time_use 统计

In [ ]:
df_rec_p = df_rec.copy()
df_rec_p.index = pd.to_datetime(df_rec_p['time'])


time_gap = np.linspace(0, 600,21)
time_gap = np.hstack([time_gap, [9999]])


df_rec_p['category'] = pd.cut(df_rec_p['rec'], time_gap)
df_rec_cout = df_rec_p['category'].value_counts()
df_rec_cout = pd.DataFrame(df_rec_cout).sort_index()
df_rec_cout = df_rec_cout.reset_index()
df_rec_cout.columns = ['category', 'count']
df_rec_cout['percent'] = df_rec_cout['count']/df_rec_cout['count'].sum()*100
df_rec_cout['category'] = df_rec_cout['category'].astype(str)


# # 绘制条形图
fig = px.bar(df_rec_cout, x='category', y='count', text='percent')
fig.update_traces(texttemplate='%{text:.2f}%', textposition='outside')
fig.update_layout(
    title=f'{df_task_p.index.min()}至{df_task_p.index.max()} 处理时间分布',
    xaxis_title='处理时间（s）',
    yaxis_title='数量占比'
)
fig.show()

## rec时间分布-02- complete-enter time 统计

In [35]:
df_task['delta_second'] = df_task['complete_time'] - df_task['enter_time']
print(df_task['enter_time'].min(),"---", df_task['enter_time'].max())
df_task['delta_second'] = df_task['delta_second'].apply(pd.Timedelta.total_seconds)
df_task.dropna(inplace=True)


delta_time_gap = np.linspace(0, 300,2)
delta_time_gap = np.hstack([delta_time_gap, [9999]])
df_task_delta_second = pd.cut(df_task['delta_second'], delta_time_gap).value_counts()
df_task_delta_second = pd.DataFrame(df_task_delta_second)
df_task_delta_second.reset_index(drop=False, inplace=True)
df_task_delta_second['percent'] = df_task_delta_second['count']/df_task_delta_second['count'].sum()
df_task_delta_second


2023-09-12 13:20:05 --- 2023-09-12 14:00:18


,delta_second,count,percent
0,"(0.0, 300.0]",806,0.85836
1,"(300.0, 9999.0]",133,0.14164
